In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mplt
import seaborn as sns
import arviz as az
import datetime

In [2]:
import fit_arima

### TO-DOs:

- Ripristinare la posterior_predictive (commentata nel codice STAN e non aggiornata per gestire le varie stazioni)
- Ripristinare la registrazione dei dati mancanti (commentata nel codice fit_arima.py e non aggiornata per gestire le varie stazioni)

## Day Planner
Il modello attualmente considerato è il seguente:

Definita la funzione $\text{sigmoide}$:
$$
\Sigma(z) = \frac{e^z - 1}{e^z + 1}
$$

Sia $s$ l'indice che scorre le stazioni:
$$
s \in \left\{ 1,\dots, S \right\}
$$
allora:
$$
\left. \mathbf{y}_s \right| \gamma_\theta, \underline{\gamma_\phi}, \sigma, \underline{\mu_\phi}, \underline{\sigma_\phi} \sim \text{ARIMA}_{2,1,1}\left( \Sigma(\gamma_{\phi,s}[0]), \Sigma(\gamma_{\phi,s}[1]), \Sigma(\gamma_\theta) , \sigma\right)\\
\left. \gamma_{\phi,s}[j] \right| \underline{\mu_\phi}, \underline{\sigma_\phi} \sim \mathcal{N}\left( \mu_{\phi}[j], \sigma_{\phi}[j] \right), \qquad j \in \left\{ 1,\dots,p \right\} = \left\{ 1,2 \right\}\\
\left. \gamma_\theta \right. \sim \mathcal{N}\left( 0,1 \right)\\
\left. \mu_\phi[j] \right. \sim \mathcal{N}\left( 0,5 \right) \qquad j \in \left\{ 1,\dots,p \right\}\\
\left. \sigma_\phi[j] \right. \sim \mathcal{IG}\left( 2.1,1.1 \right) \qquad j \in \left\{ 1,\dots,p \right\}
$$

L'ARIMA utilizza anche dati precedenti al giorno iniziale trattati come dati mancanti.
I dati mancanti in generale hanno bisogno di prior, che sono state scelte come segue:
$$
y[missing] \sim \mathcal{N}\left( 1,1 \right)
$$
(nel codice STAN sono la variabile 'w')

Tuttavia i dati precedenti i giorni iniziali sono stati trattati diversamente, secondo il modello:
$$
y_{start} | \mu_{start}, \sigma_{start} \sim \mathcal{N}\left( \mu_{start}, \sigma_{start} \right)\\
\mu_{start} \sim \mathcal{N}\left( 1,1 \right)\\
\sigma_{start} \sim \mathcal{IG}\left( 3,2 \right)
$$

Naturalmente se scrivessimo le cose per bene, la likelihood dell'ARIMA dovrebbe essere condizionata anche ai dati mancanti

In [3]:
import open_data
df = open_data.open()

Nel mio PC (br1) la porzione STAN del codice che segue è stata eseguita in ~ 26 minuti (da notare che prima di inserire le "hyper prior" che rendono il modello gerarchico, si ottenevano tempistiche di ~ 60 minuti)

(Sarebbe bello eseguire tutti il codice correggendo opportunamente in funzione dei core e segnare quanto ci vuole, quindi ad esempio con 4 core inserire catene=4 e samples_per_chain=1000, 6 core catene=6, samples = 750, il tempo da registrare sarebbe quello che appare a destra della barra di caricamento quando STAN termina la computazione)

In [4]:
ritorno = fit_arima.compute(2, 1, catene=8, samples_per_chain=500, burnin=500)

11:57:47 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

chain 5 |          | 00:00 Status

chain 6 |          | 00:00 Status

chain 7 |          | 00:00 Status

chain 8 |          | 00:00 Status

12:00:51 - cmdstanpy - ERROR - Chain [6] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [4] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [7] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [5] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [2] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [3] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [8] error: terminated by signal 2 Unknown error -2
12:00:51 - cmdstanpy - ERROR - Chain [1] error: terminated by signal 2 Unknown error -2

KeyboardInterrupt



In [ ]:
az.summary(ritorno['inference_data'], var_names=['theta','sigma','gamma_th', 'hyper_y_start_m', 'hyper_y_start_s', 'hyper_gamma_phi_m', 'hyper_gamma_phi_s'])

In [ ]:
az.plot_pair(ritorno['inference_data'], var_names=['hyper_gamma_phi_m', 'hyper_gamma_phi_s', 'theta', 'sigma'], divergences=True)

In [ ]:
az.plot_trace(ritorno['inference_data'], var_names=['phi','theta','sigma'])

In [ ]:
az.plot_posterior(ritorno['inference_data'], var_names=['theta','sigma','hyper_y_start_m','hyper_y_start_s', 'hyper_gamma_phi_m', 'hyper_gamma_phi_s'], hdi_prob=0.95)

In [ ]:
aux_shape = ritorno['inference_data'].posterior.phi.values.shape
sns.kdeplot(ritorno['inference_data'].posterior.phi.values.reshape((aux_shape[0]*aux_shape[1],aux_shape[2],aux_shape[3]))[:,0,:], legend=False)

In [ ]:
aux_shape = ritorno['inference_data'].posterior.phi.values.shape
sns.kdeplot(ritorno['inference_data'].posterior.phi.values.reshape((aux_shape[0]*aux_shape[1],aux_shape[2],aux_shape[3]))[:,1,:], legend=False)

In [ ]:
y_post_pred = ritorno['inference_data'].posterior.y_post_pred.to_numpy()
y_post_pred = y_post_pred.reshape(y_post_pred.shape[0]*y_post_pred.shape[1], y_post_pred.shape[2], y_post_pred.shape[3])
y_post_pred.shape

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display

In [ ]:
@widgets.interact(stazione = df.columns)
def f(stazione):
    col_map = sns.light_palette((20, 75, 70), input='husl', as_cmap=True)

    plt.figure(figsize=(14,8))
    ax = plt.subplot(1,1,1)

    idx_stazione = df.columns.to_list().index(stazione)
    
    """
    sns.lineplot(np.transpose(y_post_pred[0:50,:,idx_stazione]))
    """

    lower_lim = np.zeros(len(df.index))
    upper_lim = np.zeros(len(df.index))    
    for i in range(len(df.index)):
        lower_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],2.5)
        upper_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],97.5)
    
    
#    sns.lineplot(lower_lim)
#    sns.lineplot(upper_lim)
    
    for i in range(len(df.index)):
        ax.add_patch(mplt.patches.Rectangle((i,lower_lim[i]),1,upper_lim[i]-lower_lim[i], fill=True, color=col_map(180)))

    sns.lineplot(df[stazione])    
    sns.lineplot(np.mean(y_post_pred[:,:,idx_stazione], axis=0))


    index = 0
    for line in ax.get_lines():
        if index == 0:
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(155))
        else:
            col_map = sns.dark_palette((10,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
        index += 1
        
    plt.ylim(bottom=0,top=2.5)

    primo_giorno = datetime.date(2018,1,1)
    date_da_segnare = []
    date_da_segnare_posizioni = []

    for i in range(12):
        date_da_segnare.append(datetime.date(2018,i+1,1))
        date_da_segnare_posizioni.append((date_da_segnare[2*i] - primo_giorno).days)
        date_da_segnare.append(datetime.date(2018,i+1,15))
        date_da_segnare_posizioni.append((date_da_segnare[2*i+1] - primo_giorno).days)
        date_da_segnare[2*i] = date_da_segnare[2*i].isoformat()
        date_da_segnare[2*i+1] = date_da_segnare[2*i+1].isoformat()

    plt.xticks(date_da_segnare_posizioni,date_da_segnare,rotation=65)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
    plt.grid()
    
    """
    ax.get_legend().remove()
    col_vals = np.linspace(1,255,num=len(df.columns))
    index = 0
    for line in ax.get_lines():
        if(index == len(ax.get_lines()) - 1):
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            continue
        if(index == len(ax.get_lines()) - 2):
            col_map = sns.dark_palette((120,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            index += 1
            continue

        #line.set_c(col_map(int(np.round(col_vals[index]))))
        line.set_c(col_map(220))
        line.set_alpha(0.3)
        index += 1
    """
    
    

    plt.show()

In [ ]:
def f(stazione):
    col_map = sns.light_palette((20, 75, 70), input='husl', as_cmap=True)

    plt.figure(figsize=(14,8))
    ax = plt.subplot(1,1,1)

    idx_stazione = df.columns.to_list().index(stazione)
    
    """
    sns.lineplot(np.transpose(y_post_pred[0:50,:,idx_stazione]))
    """

    lower_lim = np.zeros(len(df.index))
    upper_lim = np.zeros(len(df.index))    
    for i in range(len(df.index)):
        lower_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],2.5)
        upper_lim[i] = np.percentile(y_post_pred[:,i,idx_stazione],97.5)
    
    
#    sns.lineplot(lower_lim)
#    sns.lineplot(upper_lim)
    
    for i in range(len(df.index)):
        ax.add_patch(mplt.patches.Rectangle((i,lower_lim[i]),1,upper_lim[i]-lower_lim[i], fill=True, color=col_map(180)))

    sns.lineplot(df[stazione])    
    sns.lineplot(np.mean(y_post_pred[:,:,idx_stazione], axis=0))


    index = 0
    for line in ax.get_lines():
        if index == 0:
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(155))
        else:
            col_map = sns.dark_palette((10,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
        index += 1
        
    plt.ylim(bottom=0,top=2.5)

    primo_giorno = datetime.date(2018,1,1)
    date_da_segnare = []
    date_da_segnare_posizioni = []

    for i in range(12):
        date_da_segnare.append(datetime.date(2018,i+1,1))
        date_da_segnare_posizioni.append((date_da_segnare[2*i] - primo_giorno).days)
        date_da_segnare.append(datetime.date(2018,i+1,15))
        date_da_segnare_posizioni.append((date_da_segnare[2*i+1] - primo_giorno).days)
        date_da_segnare[2*i] = date_da_segnare[2*i].isoformat()
        date_da_segnare[2*i+1] = date_da_segnare[2*i+1].isoformat()

    plt.xticks(date_da_segnare_posizioni,date_da_segnare,rotation=65)
    plt.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom=True,      # ticks along the bottom edge are off
        top=False,         # ticks along the top edge are off
        labelbottom=True)
    plt.grid()
    
    """
    ax.get_legend().remove()
    col_vals = np.linspace(1,255,num=len(df.columns))
    index = 0
    for line in ax.get_lines():
        if(index == len(ax.get_lines()) - 1):
            col_map = sns.dark_palette((230,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            continue
        if(index == len(ax.get_lines()) - 2):
            col_map = sns.dark_palette((120,90,65), input='husl', as_cmap=True)
            line.set_c(col_map(175))
            index += 1
            continue

        #line.set_c(col_map(int(np.round(col_vals[index]))))
        line.set_c(col_map(220))
        line.set_alpha(0.3)
        index += 1
    """
    
    

    plt.show()

In [ ]:
f('CAORLE')

In [ ]:
f('SAVIGNANO DI RIGO')